In [1]:
import sys,os
import numpy as np
import pylab as py
# from corelib_AA import JAMLIB

In [15]:
import sys,os
import numpy as np
import cPickle
from scipy.interpolate import RectBivariateSpline
from tools import load,BAR,logo
 
class JAMLIB(object):
  
  def __init__(self,path):
    logo()
    self.setup_alphaS(order='NLO')
    self.load_tables(path)

  # alphaS

  def setup_alphaS(self,order='NLO'):

    if   order=='LO':  self.order=0
    elif order=='NLO': self.order=1

    self.aZ  = 0.118 /(4*np.pi)
    self.mc2 = 1.43**2        
    self.mb2 = 4.3**2
    self.mZ2 = 91.187**2
    self.mt2 = 172.9**2

    self.beta=np.zeros((7,3))
    for Nf in range(3,7): 
      self.beta[Nf,0]=11.0-2.0/3.0*Nf 
      self.beta[Nf,1]=102.-38.0/3.0*Nf 
      self.beta[Nf,2]=2857.0/2.0-5033.0/18.0*Nf+325.0/54.0*Nf**2 

    self.Q20=1.0
    # uses alphaS(mZ)--> backwards evolution
    self.ab=self.evolve_a(self.mZ2,self.aZ,self.mb2,5)
    self.at=self.evolve_a(self.mZ2,self.aZ,self.mt2,5)
    self.ac=self.evolve_a(self.mb2,self.ab,self.mc2,4)
    self.a0=self.evolve_a(self.mc2,self.ac,self.Q20,3)

    # we will store all Q2 values of alphaS 
    self.storage={}

  def get_Nf(self,Q2):
    Nf=3
    if Q2>=(4.*self.mc2): Nf+=1
    if Q2>=(4.*self.mb2): Nf+=1
    if Q2>=(4.*self.mt2): Nf+=1
    return Nf

  def beta_func(self,a,Nf):
    betaf = -self.beta[Nf,0]
    if self.order>=1: betaf+=-a*self.beta[Nf,1]
    if self.order>=2: betaf+=-a*self.beta[Nf,2]
    return betaf*a**2

  def evolve_a(self,Q20,a,Q2,Nf):
    # Runge-Kutta implemented in pegasus  
    LR = np.log(Q2/Q20)/20.0
    for k in range(20):
      XK0 = LR * self.beta_func(a,Nf)
      XK1 = LR * self.beta_func(a + 0.5 * XK0,Nf)
      XK2 = LR * self.beta_func(a + 0.5 * XK1,Nf)
      XK3 = LR * self.beta_func(a + XK2,Nf)
      a+= (XK0 + 2.* XK1 + 2.* XK2 + XK3) * 0.166666666666666
    return a

  def get_a(self,Q2):

    if Q2 in self.storage:
      return self.storage[Q2]
    else:
      if self.mt2<=Q2:
        Q20,a0,Nf=self.mt2,self.at,6
      elif self.mb2<=Q2 and Q2<self.mt2: 
        Q20,a0,Nf=self.mb2,self.ab,5
      elif self.mc2<=Q2 and Q2<self.mb2: 
        Q20,a0,Nf=self.mc2,self.ac,4
      elif Q2<self.mc2:
        Q20,a0,Nf=self.mc2,self.ac,3
      a=self.evolve_a(Q20,a0,Q2,Nf)
      self.storage[Q2]=a
      return a

  def get_alphaS(self,Q2):
    return self.get_a(Q2)*4*np.pi

  # interpolation for xF

  def load_tables(self,path):
    TAB=[]
    F=os.listdir(path)
    self.npos=len(F)
    bar=BAR('loading tables',len(F))
    for f in F:
      tab=load('%s/%s'%(path,f)) 
      X=tab['X']
      Q2=tab['Q2']
      for k in tab:
        if k=='X' or k=='Q2': continue
        for kk in tab[k]:
          tab[k][kk]=RectBivariateSpline(X,Q2,tab[k][kk])
      TAB.append(tab)
      bar.next()
    bar.finish()
    self.TAB=TAB
    self.Xgrid = X
    self.Q2grid = Q2

  def get_XF(self,ipos,dist,flav,x,Q2):
    return self.TAB[ipos][dist][flav](x,Q2)[0,0]
  
  def get_Xgrid():
    return self.Xgrid
    
  def get_Q2grid():
    return self.Q2grid
  

## Loads and tests the FF

In [3]:
jamFF=JAMLIB('FFpions')

################################################ 
                                                 
     _   _    __  __ _     ___ ____              
    | | / \  |  \/  | |   |_ _| __ )             
 _  | |/ _ \ | |\/| | |    | ||  _ \             
| |_| / ___ \| |  | | |___ | || |_) |            
 \___/_/   \_\_|  |_|_____|___|____/             
                                                 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
                                                 
Authors:                                         
Nobuo Sato   nsato@jlab.org                      
Jake Ethier                                      
Wally Melnitchouk                                
Alberto Accardi                                  
################################################ 
loading tables [99%]


In [98]:
x,Q2=0.5,10.0

print 'alphaS     = ',jamFF.get_alphaS(Q2)
print 'no. of posteriors = ',jamFF.npos
print 'xFF(ipos=0) = ',jamFF.get_XF(0,'FFpion','up',x,Q2)

xgrid = jamFF.get_Xgrid()
Xmin = np.min(xgrid)
Xmax = np.max(xgrid)
print 'Xmin,Xmax: ', xmin,xmax
print xgrid 

Q2grid = jamFF.get_Q2grid()
Q2min = np.min(Q2grid)
Q2max = np.max(Q2grid)
print 'Q2min,Q2max: ', Q2min,Q2max
print Q2grid

alphaS     =  0.24654958796
no. of posteriors =  200
xF(ipos=0) =  0.290467548931
Xmin,Xmax:  1e-06 1.0
[  1.00000000e-06   1.26485522e-06   1.59985872e-06   2.02358965e-06
   2.55954792e-06   3.23745754e-06   4.09491506e-06   5.17947468e-06
   6.55128557e-06   8.28642773e-06   1.04811313e-05   1.32571137e-05
   1.67683294e-05   2.12095089e-05   2.68269580e-05   3.39322177e-05
   4.29193426e-05   5.42867544e-05   6.86648845e-05   8.68511374e-05
   1.09854114e-04   1.38949549e-04   1.75751062e-04   2.22299648e-04
   2.81176870e-04   3.55648031e-04   4.49843267e-04   5.68986603e-04
   7.19685673e-04   9.10298178e-04   1.15139540e-03   1.45634848e-03
   1.84206997e-03   2.32995181e-03   2.94705170e-03   3.72759372e-03
   4.71486636e-03   5.96362332e-03   7.54312006e-03   9.54095476e-03
   1.20679264e-02   1.52641797e-02   1.93069773e-02   2.44205309e-02
   3.08884360e-02   3.90693994e-02   4.94171336e-02   6.25055193e-02
   7.90604321e-02   1.00000000e-01   1.18000000e-01   1.36000000e-01

## Info file

In [105]:
# User-supplied  information for .info file headers
desc = 'JAMFF16 pion fragmentation functions. Returns FF(z) rather than z*FF(z). \
mem=0 => average of posteriors; mem=1-200 => FF posteriors.'
auth = "[JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A"
ref = "arXiv:1609.xxxx - e-mail accardi@jlab.org / sato@jlab.org"
ver = -1   # -1: beta  1,2,3, ... official version, and updates/debugged  ones if needed

index = 16000
    # particle=hadron to which the PDFs belong, see
    # http://pdg.lbl.gov/2015/reviews/rpp2015-rev-monte-carlo-numbering.pdf
    # For protons, neutrons, use format ±n nr nL nq1 nq2 nq3 nJ
    # E.g, proton = 2212 (u; u; d; spin 1/2 --> nJ=2*s+1)
    # For nuclei, use LZZZAAAI, see point 14 in PDG standard:
    # L nonzero is for hypernuclei, I represent isomer levels with
    # I = 0 the ground state, and Z and A should be self-explanatory.
hadron = 2212  #  2212=proton

ordQCD =1  # 1: NLO
nfl = 5

# This is for JAM FF or PDF, for others needs to change
errortype = 'MC'      # MC = Monte Carlo method --> nsamples = no. of posteriors
nsamples = jamFF.npos # HESSIAN = Hessian --> nsamples = no. free parameters
cl = 68.3             # confidence level

# Collects info from JAM fits
xgrid = jamFF.get_Xgrid()
xmin = np.min(xgrid)
xmax = np.max(xgrid)
Qgrid = jamFF.get_Q2grid()**0.5
Qmin = np.min(Qgrid)
Qmax = np.max(Qgrid)
mu = 0.
md = 0.
ms = 0.1
mc = jamFF.mc2**0.5
mb = jamFF.mb2**0.5
mt = jamFF.mt2**0.5
mZ = jamFF.mZ2**0.5

In [106]:
H=[]
H.append('SetDesc: '+desc)
H.append('SetIndex: %i'%index)
H.append('Authors: '+auth)
H.append('Reference: '+ref)
H.append('Format: lhagrid1')
H.append('DataVersion: %i'%ver)

H.append('Particle: %i'%hadron)
flavors = [-5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 21]
H.append('Flavors: '+str(flavors))
H.append('OrderQCD: %i'%ordQCD)
H.append('ForcePositive: 1')
flavscheme = 'variable' # used for any variable flavor number scheme, massive or not
H.append('FlavorScheme: '+flavscheme)
H.append('NumFlavors: %i'%nfl)

if errortype=='MC':
  nsets = nsamples+1   # number of posteriors + average
  errtype = 'replicas'  # LHAPDF-speak for MC method
elif errortype=='HESSIAN': 
  nsets = 2*nsamples+2  # 2*eigenvalues + central fit
  errtype = 'hessian'
H.append('NumMembers: %i'%nsets)
H.append('ErrorType: '+errtype)
H.append('ErrorConfLevel: %f'%cl)
H.append('XMin: %e'%xmin)
H.append('XMax: %e'%xmax)
H.append('QMin: %e'%Qmin)
H.append('QMax: %e'%Qmax)

H.append('MZ: %f'%mZ**2)
H.append('MUp: %f'%mu)
H.append('MDown: %f'%md)
H.append('MStrange: %f'%ms)
H.append('MCharm: %f'%mc)
H.append('MBottom: %f'%mb)
H.append('MTop: %f'%mt)

H.append('AlphaS_MZ: %f'%jamFF.get_alphaS(mZ**2))
H.append('AlphaS_OrderQCD: %f'%ordQCD)
H.append('AlphaS_Type: ipol')
H.append('AlphaS_Qs: '+str(Qgrid.tolist()))
alphas = []
for Q in Qgrid:
  alphas.append(jamFF.get_alphaS(Q**2))
H.append('AlphaS_Vals: '+str(alphas))
H.append('AlphaS_Lambda4: %f'%jamFF.get_alphaS(mc**2))
H.append('AlphaS_Lambda5: %f'%jamFF.get_alphaS(mb**2))
   

for l in H: print l 

SetDesc: JAMFF16 pion fragmentation functions. Returns FF(z) rather than z*FF(z). mem=0 => average of posteriors; mem=1-200 => FF posteriors.
SetIndex: 16000
Authors: [JAM collaboration] Sato N, Ethier J J, Melnitchouk W, Hirai M, Kumano S, Accardi A
Reference: arXiv:1609.xxxx - e-mail accardi@jlab.org / sato@jlab.org
Format: lhagrid1
DataVersion: -1
Particle: 2212
Flavors: [-5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 21]
OrderQCD: 1
ForcePositive: 1
FlavorScheme: variable
NumFlavors: 5
NumMembers: 201
ErrorType: replicas
ErrorConfLevel: 68.300000
XMin: 1.000000e-06
XMax: 1.000000e+00
QMin: 1.000000e+00
QMax: 3.162278e+02
MZ: 8315.068969
MUp: 0.000000
MDown: 0.000000
MStrange: 0.100000
MCharm: 1.430000
MBottom: 4.300000
MTop: 172.900000
AlphaS_MZ: 0.118000
AlphaS_OrderQCD: 1.000000
AlphaS_Type: ipol
AlphaS_Qs: [1.0, 1.2195704601594413, 1.4873521072935114, 1.8139306939110624, 2.2122162910704493, 2.697953640073, 3.2903445623126686, 4.012807031942776, 4.893900918477494, 5.9684569951223105, 7.27895

## write single .dat file

In [108]:
def writedat(ipos,hadron):
  
  if ipos==0: 
    type='central'
  else:
    type='replica'

  H = []
  H.append('PdfType: '+type)
  H.append('Format: lhagrid1')
  H.append('---')  
  
  for l in H: print l
  
  xvals = ["%10.4e" % x for x in xgrid]
  print ' '.join(xvals)
  Qvals = ["%10.4e" % Q for Q in Qgrid]
  print ' '.join(Qvals)
  flavs = ["%2i" % f for f in flavors]
  print ' '.join(flavs)
    
  partons = ['bp','cp','sp','up','dp','dp','up','sp','cp','bp','g']
  npart = len(partons)
  
  full_dist = []
  ix = 0
  iQ = 0
  il = 0
  for x in xgrid:
    for Q in Qgrid:
      if ipos==0:  # Avreraged PDF
        print "******** Average PDF values still to be coded! *********** "
      else:   # ipos-th posterior 
        FF = [jamFF.get_XF(ipos-1,hadron,f,x,Q**2) for f in partons]   # <--------- NOTE: for now x*FF(x), Shall we distribute FF(x) as for other PDFs ??  
        if Q < mb: FF[0] = FF[9] = 0
        if Q < mc: FF[1] = FF[8] = 0
      FF.append([x,Q])
      full_dist.append(FF)
  template = ''.join(['{:12.5e} ' for n in range(npart)])    # <----- how many sig digits ?
  for l in full_dist: 
    print template.format(l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[9],l[10]),":",l[11]  # <--- horrible but functional python 
        
    
hadron = 'FFpion'
writedat(1,hadron)

PdfType: replica
Format: lhagrid1
---
1.0000e-06 1.2649e-06 1.5999e-06 2.0236e-06 2.5595e-06 3.2375e-06 4.0949e-06 5.1795e-06 6.5513e-06 8.2864e-06 1.0481e-05 1.3257e-05 1.6768e-05 2.1210e-05 2.6827e-05 3.3932e-05 4.2919e-05 5.4287e-05 6.8665e-05 8.6851e-05 1.0985e-04 1.3895e-04 1.7575e-04 2.2230e-04 2.8118e-04 3.5565e-04 4.4984e-04 5.6899e-04 7.1969e-04 9.1030e-04 1.1514e-03 1.4563e-03 1.8421e-03 2.3300e-03 2.9471e-03 3.7276e-03 4.7149e-03 5.9636e-03 7.5431e-03 9.5410e-03 1.2068e-02 1.5264e-02 1.9307e-02 2.4421e-02 3.0888e-02 3.9069e-02 4.9417e-02 6.2506e-02 7.9060e-02 1.0000e-01 1.1800e-01 1.3600e-01 1.5400e-01 1.7200e-01 1.9000e-01 2.0800e-01 2.2600e-01 2.4400e-01 2.6200e-01 2.8000e-01 2.9800e-01 3.1600e-01 3.3400e-01 3.5200e-01 3.7000e-01 3.8800e-01 4.0600e-01 4.2400e-01 4.4200e-01 4.6000e-01 4.7800e-01 4.9600e-01 5.1400e-01 5.3200e-01 5.5000e-01 5.6800e-01 5.8600e-01 6.0400e-01 6.2200e-01 6.4000e-01 6.5800e-01 6.7600e-01 6.9400e-01 7.1200e-01 7.3000e-01 7.4800e-01 7.6600e-01 7.840